In [ ]:
from qtensor import Info, MPS #, Gates, CircuitCXFid
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
from torch import optim

N = 5

info = Info()

# Тест для генерации E

In [ ]:
from qtensor import MPO

mps = MPS(info)
mps.get_random_mps(N, 1)
mpo = MPO(info)
mpo.pure_state(mps)

### проверка совпадение

In [ ]:

indices = []
def gen(pref):
    if len(pref) == N:
        global indices
        indices.append(pref)
        return
    for i in range(2):
        gen(pref + [i])
        
gen([])
for i1 in indices:
    for i2 in indices:
        val1 = mps.get_element(i1)
        val2 = mps.get_element(i2)
        corr = torch.conj(val1) * val2
        our = mpo.get_element(i1, i2)
        diff = torch.abs(corr - our)
        assert diff.item() < 1e-12

### проверка trace

In [ ]:
mpo.get_trace()

# Произведение

In [ ]:

def gen_E():
    info = Info()
    mps = MPS(info)
    
    mps.get_random_mps(N, 2)

    mpo = MPO(info)
    mpo.all_zeros_state(N)
    mpo.pure_state(mps)
    return mpo

mpo1 = gen_E()
mpo2 = gen_E()
A1 = mpo1.get_full_matrix()
A2 = mpo2.get_full_matrix()
B = mpo1.get_product(mpo2).get_full_matrix()
assert torch.sum(torch.abs((A1 @ A2) - B)) < 1e-12

# Генерация случайной MPO

In [ ]:
mpo = MPO(info)

mpo.random_full(N, 2)

In [ ]:
mpo.get_trace()

In [ ]:
mpo.r

In [ ]:
A = mpo.get_full_matrix()
print(A.trace())

In [ ]:
mpo_T = mpo.transpose()
AT = mpo_T.get_full_matrix()

assert torch.sum(torch.abs(A.T - AT)) < 1e-12

# Генерация случайной положительно определенной rho

In [ ]:
mpo = MPO(info)
mpo.random_rho(N, 2)

In [ ]:
mpo.get_trace()

In [ ]:
mpo.r

In [ ]:
mpo1 = MPO(info)
mpo1.random_rho(N, 2)
mpo2 = MPO(info)
mpo2.random_rho(N, 2)
A1 = mpo1.get_full_matrix()
A2 = mpo2.get_full_matrix()
B = mpo1.get_product(mpo2).get_full_matrix()
assert torch.sum(torch.abs((A1 @ A2) - B)) < 1e-12

## Проверка get_product_trace

In [ ]:
E = gen_E()
rho = MPO(info)
rho.random_rho(N, 5);


In [ ]:
a = rho.get_product(E)

In [ ]:
a.get_trace()

In [ ]:
rho.get_product_trace(E)

## Оптимизация

In [ ]:
import time

def optimize(list_E: list, values: list, N: int, R: int, eps: float):
    rho = MPO(info)
    rho.random_rho(N, R)
    [ x.requires_grad_(True) for x in rho.tt_cores]
    optimizer = optim.Adam(rho.tt_cores)
    it = 0
    f = torch.tensor(0 + 0.j, device=info.device)
    f_prev = 1.0
    k = 0
    while(abs(f.item()) > eps or it == 0):
        t1 = time.time()
        optimizer.zero_grad()
        f = torch.tensor(0 + 0.j, device=info.device)
        for n, E in enumerate(list_E):
            z = (rho.get_product_trace(E) - values[n])
            f += torch.conj(z) * z
            #f += torch.abs(z)
        f = f / len(values)

        t2 = time.time()
        
        f.backward()
        optimizer.step()
        
        t3 = time.time()

        print(f'\rIteration {it}: MSE={f.item()}, t_calc={t2-t1}, t_backward={t3-t2}', end = '')

        k = k + 1 if f.item() == f_prev else 0
        f_prev = f.item()
        it += 1
        if (k == 20):
            break
    return rho

In [ ]:
list_E = [gen_E() for i in range(1000)]
rho_ideal = MPO(info)
rho_ideal.random_rho(N, 1)
values = [rho_ideal.get_product_trace(E) for E in list_E]

In [ ]:
rho = optimize(list_E, values, N, 1, 1e-8)

In [ ]:
with torch.no_grad():
    f_prov = torch.tensor(0 + 0.j, device=info.device)
    for i in range(10000):
        E_prov = gen_E()
        z = rho.get_product_trace(E_prov) - rho_ideal.get_product_trace(E_prov)
        f_prov += torch.conj(z) * z
    f_prov /= 10000
    print(f_prov)  

In [ ]:
E = gen_E()
torch.abs(rho.get_product_trace(E) - rho_ideal.get_product_trace(E)) ** 2

In [ ]:
rho.r

In [ ]:
rho_ideal.r

In [ ]:
f_prov

In [ ]:
rho.get_trace()

In [ ]:
np.mean(np.array(values))